[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TMDU-AI/AI-yodosha/blob/main/Chapter5/chap5.ipynb)

# **5章　肺のX線画像を用いた画像分類にトライしよう**

# 5-1　外部のデータを読み込む

まず事前準備としてGoogle Driveをマウントします。  
ファイルアイコン（フォルダのマーク）をクリックしてマウントのアイコンをクリックしてください。  
Colabからコード5-0を実行するように促された場合は実行します。  
時間がかかる場合があります。  
driveフォルダが表示されない場合、ファイルアイコンを再度クリックし直して表示し直すと表示されていることが多いようです。

コード5-0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

コード5-1-1の前にGoogle DriveのMyDriveに「images_TMDU.zip」がアップロードされていることを確認してください。  
Colabからのアップロードは時間がかかることがあります。  
その場合はGoogle Drive（https://drive.google.com/drive/home ）から直接アップロードしてみてください。  

コード5-1-1　images_TMDU.zipの解凍

In [ ]:
!unzip '/content/drive/MyDrive/images_TMDU.zip'

# 5-2　デジタル画像のデータは数値で表すことができる

コード5-2-1　画像「test.jpg」の読み込みと図示

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
test = load_img('/content/images/test.jpg', color_mode='rgb')
plt.imshow(test)
plt.show()
print(test)

コード5-2-2　画像サイズとカラーモードの変換

In [ ]:
test2 = load_img(
    '/content/images/test.jpg', color_mode='grayscale',
    target_size=(10, 10))
plt.imshow(test2, 'gray')
plt.show()
print(test2)

コード5-2-3　画像データを配列データに変換

In [ ]:
from keras.preprocessing.image import img_to_array
test2_img = img_to_array(test2)
print(type(test2_img))
print(test2_img)
print(test2_img.shape)

コード5-2-4　画像サイズ(10, 10)，カラーでの図示と配列変換

In [ ]:
test3 = load_img(
    '/content/images/test.jpg', color_mode='rgb', target_size=(10, 10))
plt.imshow(test3)
plt.show()
test3_img = img_to_array(test3)
print(test3_img)
print(test3_img.shape)

コード5-2-5　インデックスによるデータへのアクセス

In [ ]:
# (10, 10, 3)は，10行あり，その各行は10列あり，その各列に3つの色要素が格納
print(len(test3_img))       # 「test3_img」の値の長さ（要素数；行数）を表示
print(test3_img[0])         # その0番行「test3_img[0]」の値を表示
print(len(test3_img[0]))    # その0番行の値の長さ（要素数；列数）を表示
print(test3_img[0][0])      # その0番行，0番列「test3_img[0][0]」の値を表示
print(len(test3_img[0][0])) # その値の長さ（要素数）を表示
print(test3_img[0][0][0])   # その0番行，0番列，0番要素（Red）の値を表示

# 5-3　肺のX線画像の分類モデルを作成してみよう

**STEP1　肺のX線画像ファイルのリスト作成**

コード5-3-1　ファイル名のリストを作成

In [ ]:
import os
list_healthy = [
    i for i in os.listdir('/content/images/COVID-NORMAL/healthy')
    if not i.startswith('.')]
list_covid19 = [
    i for i in os.listdir('/content/images/COVID-NORMAL/covid19')
    if not i.startswith('.')]
print(len(list_healthy))
print(len(list_covid19))

コード5-3-2　リスト内包表記の例１

In [ ]:
example1 = [i for i in range(5)]
print(example1)
temp = [1, 3, 5, 7, 9]
example2 = [i*2 for i in temp]
print(example2)

コード5-3-3　リスト内包表記の例２

In [ ]:
temp = [1, 3, 5, 7, 9]
example3 = [i*2 for i in temp if i > 4]
print(example3)
example4 = [i*2 for i in temp if not i > 5]
print(example4)

コード5-3-4　リスト内包表記の例３

In [ ]:
temp2 = ['須藤', '佐藤', '加藤', '佐川']
example5 = [name for name in temp2 if name.startswith('佐')]
print(example5)
example6 = [name for name in temp2 if not name.endswith('藤')]
print(example6)

コード5-3-5　os.listdir()の確認１

In [ ]:
import os
os.listdir('/content/images/COVID-NORMAL')

コード5-3-6　os.listdir()の確認２

In [ ]:
os.listdir('/content/images/COVID-NORMAL/healthy')

**STEP2　画像ファイル数の集計と変数の作成**

コード5-3-7　画像ファイル数の集計

In [ ]:
num_healthy = len(list_healthy)
num_covid19 = len(list_covid19)
num_all = num_healthy + num_covid19
print(num_all)

コード5-3-8　画像データ格納用NumPy配列の作成

In [ ]:
import numpy as np
images_temp = np.zeros((num_all, 64, 64, 1), dtype=float)
labels_temp = np.zeros((num_all, 1), dtype=int)
print(images_temp.shape)
print(labels_temp.shape)

コード5-3-9　画像データ格納用NumPy配列の値の確認

In [ ]:
print(labels_temp)
print(images_temp[0])

**STEP3　健康な肺のX線画像の読み込み**

コード5-3-10　画像の読み込み（list_healthy）



In [ ]:
path_healthy = '/content/images/COVID-NORMAL/healthy'
for i in range(num_healthy):
  file = f'{path_healthy}/{list_healthy[i]}'
  file_img = load_img(
      file, color_mode='grayscale', target_size=(64, 64),
      interpolation='lanczos')
  images_temp[i] = img_to_array(file_img) / 255
  labels_temp[i] = 0

コード5-3-11　「f'{変数名}'」の例

In [ ]:
name = '須藤'
print('name')
print(f'{name}')
print('私の名前はnameです')
print(f'私の名前は{name}です')

コード5-3-12　images_tempの変換後の値

In [ ]:
images_temp[0]

**STEP4　肺炎のX線画像の読み込み**

コード5-3-13　画像の読み込み（list_covid19）

In [ ]:
path_covid19 = '/content/images/COVID-NORMAL/covid19'
for i in range(num_covid19):
  file = f'{path_covid19}/{list_covid19[i]}'
  file_img = load_img(
      file, color_mode='grayscale', target_size=(64, 64),
      interpolation='lanczos')
  images_temp[i + num_healthy] = img_to_array(file_img) / 255
  labels_temp[i + num_healthy] = 1

コード5-3-14　images_tempとlabels_tempの変換後の値

In [ ]:
print(images_temp[116])
print(labels_temp[116])

**STEP5　X線画像のシャッフル**

コード5-3-15　0から231までの数字の配列を作成

In [ ]:
num_list = np.arange(num_all)
print(num_list)

コード5-3-16　NumPy配列をシャッフルする

In [ ]:
np.random.seed(111)
np.random.shuffle(num_list)
print(num_list)

コード5-3-17　images_tempとlabels_tempの配列をシャッフルする

In [ ]:
print(labels_temp[0:10])
x_train = images_temp[num_list]
y_train = labels_temp[num_list]
print(y_train[0:10])
print(x_train.shape)
print(y_train.shape)

コード5-3-18　NumPy配列の要素を並べ替える

In [ ]:
array1 = np.array([10, 20, 30, 40, 50])
print(array1)
order = [3, 2, 0, 4, 1]
array2 = array1[order]
print(array2)

**STEP6　深層学習モデルの作成**

コード5-3-19　深層学習モデルの作成

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()
model.add(Flatten(input_shape=(64, 64, 1))) # 配列を1次元に変換
model.add(Dense(512, activation='relu'))  # 中間層1層目を作成
model.add(Dense(256, activation='relu'))  # 中間層2層目を作成
model.add(Dense(128, activation='relu'))  # 中間層3層目を作成
model.add(Dense(1, activation='sigmoid')) # 出力層を作成
model.compile(
    loss='binary_crossentropy', optimizer='Adam',
    metrics=['accuracy']) # 学習方法の設定
model.summary()

コード5-3-20　学習の実施

In [ ]:
result = model.fit(
    x_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

コード5-3-21　学習過程の図示

In [ ]:
import matplotlib.pyplot as plt
plt.plot(result.history['accuracy'], label='accuracy')
plt.plot(result.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()
plt.plot(result.history['loss'], label='loss')
plt.plot(result.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

コード5-3-22　Dropoutを加えて過学習を防ぐ

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout # Dropout関数を追加

model = Sequential()
model.add(Flatten(input_shape=(64, 64, 1)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5)) # Dropoutの層を追加
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5)) # Dropoutの層を追加
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()
result = model.fit(
    x_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

コード5-3-23　Dropoutを加えた学習過程の図示

In [ ]:
import matplotlib.pyplot as plt
plt.plot(result.history['accuracy'], label='accuracy')
plt.plot(result.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()
plt.plot(result.history['loss'], label='loss')
plt.plot(result.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# 5-4　未知のデータが肺炎かどうかを予測

コード5-4-1　新たな画像での分類

In [ ]:
img1 = img_to_array(load_img(
    '/content/images/covid.jpg',
    color_mode='grayscale', target_size=(64, 64))) / 255
img2 = img_to_array(load_img(
    '/content/images/NORMAL.jpg',
    color_mode='grayscale', target_size=(64, 64))) / 255
check = np.zeros((2, 64, 64, 1))
check[0] = img1
check[1] = img2
prob = model.predict(check)
print(prob)